<a href="https://colab.research.google.com/github/gopal2812/mlblr/blob/master/lstmassignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import sys
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from random import randint
from keras.models import load_model
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
!ls

sample_data


In [3]:
from google.colab import files
uploaded = files.upload()

Saving wonderland.txt to wonderland.txt


In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

2. Remove all the punctuation from the source text

In [0]:
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
  
 	# replace '\n' with a space ' '
	#doc = doc.replace('\n', ' ')
  
	# split into tokens by full stop
	tokens = doc.split('\n')
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# make lower case
	tokens = [word.lower() for word in tokens]
	# using join() +  split() to 
	# perform removal of empty list
	for word in tokens:
		word = ' '.join(word).split()
	tokens = list(filter(None, tokens))
	print(type(tokens))
	return tokens

In [0]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [7]:
in_filename = "wonderland.txt"
doc = load_doc(in_filename)
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

<class 'list'>
['alices adventures in wonderland', 'lewis carroll', 'the millennium fulcrum edition 30', 'chapter i down the rabbithole', 'alice was beginning to get very tired of sitting by her sister on the', 'bank and of having nothing to do once or twice she had peeped into the', 'book her sister was reading but it had no pictures or conversations in', 'it and what is the use of a book thought alice without pictures or', 'conversations', 'so she was considering in her own mind as well as she could for the', 'hot day made her feel very sleepy and stupid whether the pleasure', 'of making a daisychain would be worth the trouble of getting up and', 'picking the daisies when suddenly a white rabbit with pink eyes ran', 'close by her', 'there was nothing so very remarkable in that nor did alice think it so', 'very much out of the way to hear the rabbit say to itself oh dear', 'oh dear i shall be late when she thought it over afterwards it', 'occurred to her that she ought to have wondere

3. Train the model on padded sequences (Links to an external site.) rather than random sequences of characters. 

In [9]:
#We would be taking a compromise so we save a lot of memory and have to truncate maybe 5% of all the sequences. 
#Sequences that are too long should be truncated at the end
num_lines = [len(line) for line in tokens]
std_line = np.std(num_lines)
mean_line = np.mean(num_lines)
seq_len= (int)((mean_line+2*std_line))

print(mean_line, std_line, seq_len, np.median(num_lines))

#also find the most frequent length
(values,counts) = np.unique(num_lines,return_counts=True)
ind=np.argmax(counts)
print(values[ind])

seq_length=100

tokenizer = Tokenizer()


def get_sequence_of_tokens(tokens):
    ## tokenization
    tokenizer.fit_on_texts(tokens)
    vocab_size = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in tokens:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, vocab_size

sequences, vocab_size = get_sequence_of_tokens(tokens)
print(sequences[:200])




53.63508064516129 18.977711332319586 91 64.0
67
[[303, 467], [303, 467, 11], [303, 467, 11, 843], [1481, 1482], [1, 1483], [1, 1483, 1484], [1, 1483, 1484, 1485], [1, 1483, 1484, 1485, 1486], [304, 9], [304, 9, 37], [304, 9, 37, 1], [304, 9, 37, 1, 689], [10, 13], [10, 13, 251], [10, 13, 251, 3], [10, 13, 251, 3, 101], [10, 13, 251, 3, 101, 27], [10, 13, 251, 3, 101, 27, 468], [10, 13, 251, 3, 101, 27, 468, 7], [10, 13, 251, 3, 101, 27, 468, 7, 346], [10, 13, 251, 3, 101, 27, 468, 7, 346, 76], [10, 13, 251, 3, 101, 27, 468, 7, 346, 76, 16], [10, 13, 251, 3, 101, 27, 468, 7, 346, 76, 16, 415], [10, 13, 251, 3, 101, 27, 468, 7, 346, 76, 16, 415, 18], [10, 13, 251, 3, 101, 27, 468, 7, 346, 76, 16, 415, 18, 1], [844, 2], [844, 2, 7], [844, 2, 7, 347], [844, 2, 7, 347, 129], [844, 2, 7, 347, 129, 3], [844, 2, 7, 347, 129, 3, 52], [844, 2, 7, 347, 129, 3, 52, 130], [844, 2, 7, 347, 129, 3, 52, 130, 55], [844, 2, 7, 347, 129, 3, 52, 130, 55, 589], [844, 2, 7, 347, 129, 3, 52, 130, 55, 589, 5]

In [0]:
def generate_padded_sequences(input_sequences, seq_length):
    #max_sequence_len = max([len(i) for i in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length, padding='pre',truncating='pre'))
    X, y = input_sequences[:,:-1],input_sequences[:,-1]
    y = to_categorical(y, num_classes=vocab_size)
    return X, y

X, y = generate_padded_sequences(sequences, seq_length)

5. Add dropout to the input layer, remove it from the layer before dense layer. Use Dropout value of 0.1 everywhere

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length-1, mask_zero=True))
model.add(Dropout(0.1))
#A dropout on the input means that for a given probability, the data on the input 
# connection to each LSTM block will be excluded from node activation and weight updates.
model.add(LSTM(256, input_shape=(seq_length, vocab_size), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0727 10:08:04.228968 140038774474624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 10:08:04.266468 140038774474624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 10:08:04.273483 140038774474624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 10:08:04.385874 140038774474624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 10:08:04.393807 

In [12]:
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 99, 50)            132350    
_________________________________________________________________
dropout_1 (Dropout)          (None, 99, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 99, 256)           314368    
_________________________________________________________________
dropout_2 (Dropout)          (None, 99, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 2647)              680279    
Total params: 1,652,309
Trainable params: 1,652,309
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [earlystop]

4. Train the model for 100 epochs

In [14]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
24080/24080 [==============================] - 74s 3ms/step - loss: 6.4714
Epoch 2/100


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


24080/24080 [==============================] - 69s 3ms/step - loss: 6.0424
Epoch 3/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.9621
Epoch 4/100
24080/24080 [==============================] - 70s 3ms/step - loss: 5.8468
Epoch 5/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.7501
Epoch 6/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.6413
Epoch 7/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.5336
Epoch 8/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.4324
Epoch 9/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.3429
Epoch 10/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.2671
Epoch 11/100
24080/24080 [==============================] - 69s 3ms/step - loss: 5.1995
Epoch 12/100
24080/24080 [==============================] - 68s 3ms/step - loss: 5.1361
Epoch 13/100
24080/24080 [==========================

In [15]:
!ls

sample_data  wonderland.txt


In [0]:
#model = load_model('weights-improvement-78-6.2132-bigger.hdf5')

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	char_gen=0
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded],
                            maxlen=seq_length-1,
                            padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		#print(yhat.shape, yhat)
    # map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			#print("Index:", index)
			if index == yhat:
				out_word = word
				break
		# append to input
		#print(out_word)
		in_text += ' ' + out_word
		char_gen+=len(out_word)
		if(char_gen >= 500):
				print("completed 500 character")
				break
		result.append(out_word)
	return ' '.join(result)

In [21]:
print(tokens[0])
# select a seed text
seed_text = tokens[randint(0,len(tokens))]
print(seed_text + '\n')

alices adventures in wonderland
out of its waistcoatpocket and looked at it and then hurried on



In [22]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 500)
print(generated)

completed 500 character
it and the white rabbit had had finished this young lady is out of sight and she was a very truthful cake on among the distant little thing howled so at last and looked at her for one and she had a little way with it and the little little thing howled so alice did what was no use in spite of sight of the same age the queen was sitting on the fan and she went on what she found the same age as she was a little small terrier which had no idea what that she had wept anything to see the same little thing was thinking and the door was the door was to get out what it was no label this time and the door was to sing a morsel of the song
